In [1]:
# Copiez ce code À LA FIN de votre fichier, après le contenu existant
# =============================================================================

import unicodedata

# =============================================================================
# CORRECTION ASCII POUR PINECONE IDs
# =============================================================================

def sanitize_pinecone_id(text: str) -> str:
    """
    Convertit un texte en ID valide pour Pinecone (ASCII uniquement).
    
    Résout les erreurs comme:
    - 'visual_table_Tableau_VII-7_Proportion_de_célibataires_définitif_p5_t7'
    - 'visual_chart_Graphique_IV-16_Pourcentage_de_la_déclaration_de_p_p38_i2'
    
    Args:
        text: Texte source pouvant contenir des caractères non-ASCII
        
    Returns:
        ID valide pour Pinecone (ASCII uniquement)
        
    Examples:
        >>> sanitize_pinecone_id("visual_table_Tableau_VII-7_Proportion_de_célibataires_définitif_p5_t7")
        'visual_table_Tableau_VII_7_Proportion_de_celibataires_definitif_p5_t7'
        
        >>> sanitize_pinecone_id("visual_chart_Graphique_IV-16_Pourcentage_de_la_déclaration_de_p_p38_i2")
        'visual_chart_Graphique_IV_16_Pourcentage_de_la_declaration_de_p_p38_i2'
    """
    if not text:
        return f"item_{hash('empty') % 1000000}"
    
    # 1. Normalisation Unicode (é → e, à → a, ç → c, etc.)
    text = unicodedata.normalize('NFD', text)
    text = ''.join(c for c in text if unicodedata.category(c) != 'Mn')
    
    # 2. Remplacer tous les caractères non-ASCII par underscore
    text = re.sub(r'[^a-zA-Z0-9_-]', '_', text)
    
    # 3. Nettoyer les underscores multiples
    text = re.sub(r'_+', '_', text)
    
    # 4. Supprimer les underscores en début/fin
    text = text.strip('_')
    
    # 5. Limiter la longueur (Pinecone max 512, on prend 80 pour sécurité)
    if len(text) > 80:
        text = text[:80].rstrip('_')
    
    # 6. S'assurer qu'on a quelque chose de valide
    if not text or not text.replace('_', '').replace('-', ''):
        text = f"item_{hash(str(text)) % 1000000}"
    
    return text


def test_sanitize_pinecone_id():
    """Teste la fonction de sanitisation avec des cas réels."""
    
    test_cases = [
        # Cas problématiques réels de votre système
        "visual_table_Tableau_VII-7_Proportion_de_célibataires_définitif_p5_t7",
        "visual_chart_Graphique_IV-16_Pourcentage_de_la_déclaration_de_p_p38_i2",
        "visual_chart_Graphique_I-10_Évolution_du_rapport_de_masculinité_p27_i1",
        
        # Autres cas à tester
        "Chapitre 1- ETAT-STRUCTURE-POPULATION-Rapport-Provisoire-RGPH5_juillet2024_0_p28_i1",
        "Graphique_I-11_Pyramide_des_âges_de_la_population_p29_i1",
        "Chapitre 4 - FECONDITE-NATALITE-Rapport-Provisoire-RGPH5_juillet2024_0_p25_i2",
        
        # Cas limites
        "",
        "___---___",
        "a" * 200,  # Trop long
    ]
    
    print("🧪 TEST DE SANITISATION PINECONE IDS")
    print("=" * 80)
    
    for i, test_id in enumerate(test_cases, 1):
        fixed_id = sanitize_pinecone_id(test_id)
        
        print(f"\nTest {i}:")
        print(f"  Avant  : {test_id}")
        print(f"  Après  : {fixed_id}")
        print(f"  ASCII  : {fixed_id.isascii()}")
        print(f"  Longueur: {len(fixed_id)}")
        print(f"  Valide : {'✅' if fixed_id.isascii() and len(fixed_id) <= 80 and fixed_id else '❌'}")



In [4]:
test_sanitize_pinecone_id()

🧪 TEST DE SANITISATION PINECONE IDS


NameError: name 're' is not defined

In [ ]:
register_image(
    source_png="vos_rapport/rgph/261a7100-dfa2-4754-ace7-f4424f537c17.png",
    pdf_file="RGPH-5-2023",
    page=14,
    image_index=1,
    caption="Répartition des occupés par secteur institutionnel selon le secteur d’activités"
)

In [5]:
# shared/utils.py
"""
Utilitaires partagés pour l'extraction d'images (graphiques) et de tables depuis les PDF
et création des index CSV correspondants avec légendes.
"""
import csv
import io
import re
from pathlib import Path

from PyPDF2 import PdfReader
from PyPDF2.generic import IndirectObject
from PIL import Image
import camelot  # pip install camelot-py[cv]

# Dossiers et fichiers fixes à la racine
IMAGES_DIR = Path("./images")
INDEX_IMAGES_CSV = Path("./charts_index.csv")
TABLES_DIR = Path("./tables")
INDEX_TABLES_CSV = Path("./tables_index.csv")


def extract_captions(text: str, prefix: str) -> list[str]:
    """
    Extrait les légendes commençant par prefix (ex. 'Graphique', 'Tableau')
    au début d'une ligne.
    """
    # Pattern multiline, début de ligne, capture jusqu'à fin de ligne
    pattern = re.compile(
        rf"^{prefix}\s+[\w\-]+\s*:\s*.+$",
        re.IGNORECASE | re.MULTILINE
    )
    return pattern.findall(text)


def extract_images(pdf_path: Path) -> list[tuple[int, str, Path, str]]:
    """
    Extrait les images (graphiques) d'un PDF en ignorant la première et la dernière image
    (entête/pied de page), associe la légende et nomme le fichier selon cette légende.
    Retourne: [(page, image_name, image_path, caption)].
    """
    images: list[tuple[int, str, Path, str]] = []
    reader = PdfReader(str(pdf_path))
    for page_num, page in enumerate(reader.pages, start=1):
        page_text = page.extract_text() or ""
        captions = extract_captions(page_text, "Graphique")
        xobjs = page.get("/Resources", {}).get("/XObject", {})
        if not isinstance(xobjs, dict):
            continue
        items = list(xobjs.items())
        # Contenu utile sans entête/pied
        content = items[1:-1] if len(items) > 2 else items
        for idx, (name_key, ref) in enumerate(content, start=1):
            try:
                obj = ref.get_object() if isinstance(ref, IndirectObject) else ref
                if obj.get("/Subtype") == "/Image":
                    data = obj.get_data()
                    img = Image.open(io.BytesIO(data))
                    if img.mode != "RGB":
                        img = img.convert("RGB")
                    # Légende correspondante
                    caption = captions[idx-1] if idx-1 < len(captions) else ""
                    # Fichier nommé d'après la légende
                    safe = re.sub(r"[^\w\- ]", "", caption).strip().replace(" ", "_")
                    filename = f"{safe or pdf_path.stem}_p{page_num}_i{idx}.png"
                    out_path = IMAGES_DIR / filename
                    out_path.parent.mkdir(parents=True, exist_ok=True)
                    img.save(out_path)
                    images.append((page_num, filename, out_path, caption))
            except Exception:
                continue
    return images


def extract_tables(pdf_path: Path) -> list[tuple[int, int, Path, str]]:
    """
    Extrait les tableaux d'un PDF via Camelot, associe leur légende et nomme
    le fichier CSV selon la légende.
    Retourne: [(page, table_idx, table_path, caption)].
    """
    tables: list[tuple[int, int, Path, str]] = []
    reader = PdfReader(str(pdf_path))
    # Captions par page
    texts = [p.extract_text() or "" for p in reader.pages]
    caps_map = {i+1: extract_captions(texts[i], "Tableau") for i in range(len(texts))}
    TABLES_DIR.mkdir(parents=True, exist_ok=True)
    for flavor in ("lattice", "stream"):
        try:
            found = camelot.read_pdf(str(pdf_path), flavor=flavor, pages="all")
            for idx, table in enumerate(found, start=1):
                page = int(table.page)
                captions = caps_map.get(page, [])
                caption = captions[idx-1] if idx-1 < len(captions) else ""
                safe = re.sub(r"[^\w\- ]", "", caption).strip().replace(" ", "_")
                filename = f"{safe or pdf_path.stem}_p{page}_t{idx}.csv"
                out_path = TABLES_DIR / filename
                out_path.parent.mkdir(parents=True, exist_ok=True)
                table.to_csv(str(out_path))
                tables.append((page, idx, out_path, caption))
        except Exception:
            continue
    return tables


def generate_charts_index(pdf_source: Path) -> None:
    """
    Extrait et indexe tous les graphiques et tableaux d'un PDF ou dossier de PDFs.
    Génère deux CSV:
      - charts_index.csv (image_id, pdf_path, page, image_path, caption)
      - tables_index.csv (table_id, pdf_path, page, table_path, caption)
    """
    IMAGES_DIR.mkdir(parents=True, exist_ok=True)
    TABLES_DIR.mkdir(parents=True, exist_ok=True)
    with INDEX_IMAGES_CSV.open("w", newline="", encoding="utf-8") as imgf, \
         INDEX_TABLES_CSV.open("w", newline="", encoding="utf-8") as tblf:
        iw = csv.writer(imgf)
        iw.writerow(["image_id", "pdf_path", "page", "image_path", "caption"])
        tw = csv.writer(tblf)
        tw.writerow(["table_id", "pdf_path", "page", "table_path", "caption"])
        paths = ([pdf_source] if pdf_source.is_file() else list(pdf_source.rglob("*.pdf")))
        for pdf in paths:
            for page, name, path, cap in extract_images(pdf):
                iw.writerow([name.rsplit('.',1)[0], str(pdf), page, str(path), cap])
            for page, idx, path, cap in extract_tables(pdf):
                tw.writerow([path.stem, str(pdf), page, str(path), cap])
    print(f"[✔] Images indexées -> {INDEX_IMAGES_CSV}")
    print(f"[✔] Tables indexées -> {INDEX_TABLES_CSV}")

In [3]:
# shared/utils.py
"""
Utilitaires partagés pour l'extraction d'images (graphiques) et de tables depuis les PDF
et création des index CSV correspondants avec légendes.
Version améliorée avec PyPDF2 uniquement - plus stable.
"""
import csv
import io
import re
from pathlib import Path
import logging

from PyPDF2 import PdfReader
from PyPDF2.generic import IndirectObject
from PIL import Image

# Dossiers et fichiers fixes à la racine
IMAGES_DIR = Path("./images")
INDEX_IMAGES_CSV = Path("./charts_index.csv")

# Configuration du logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


def extract_title_from_caption(caption: str) -> str:
    """
    Extrait le titre principal d'une légende en supprimant le préfixe et les caractères spéciaux.
    Ex: "Graphique 2 : Rapport de masculinité de la population" -> "Rapport_de_masculinite_de_la_population"
    """
    if not caption:
        return ""
    
    # Supprimer le préfixe (Graphique X:, Tableau Y:, etc.)
    title = re.sub(r'^(Graphique|Tableau)\s+[\w\-]+\s*:\s*', '', caption, flags=re.IGNORECASE)
    
    # Nettoyer et formater pour nom de fichier
    title = re.sub(r'[^\w\s\-]', '', title)  # Garder lettres, chiffres, espaces, tirets
    title = re.sub(r'\s+', '_', title.strip())  # Remplacer espaces par underscores
    title = title.lower()  # Minuscules pour cohérence
    
    return title[:50]  # Limiter la longueur


def extract_captions(text: str, prefix: str) -> list[str]:
    """
    Extrait les légendes commençant par prefix (ex. 'Graphique', 'Tableau')
    au début d'une ligne avec pattern amélioré.
    """
    if not text:
        return []
    
    # Pattern plus robuste pour capturer les légendes complètes
    pattern = re.compile(
        rf"^{prefix}\s+[\w\-]+\s*:\s*.+?(?=\n\n|\n[A-Z]|\n{prefix}|\Z)",
        re.IGNORECASE | re.MULTILINE | re.DOTALL
    )
    matches = pattern.findall(text)
    
    # Nettoyer les matches (supprimer retours à la ligne internes)
    cleaned_matches = []
    for match in matches:
        cleaned = re.sub(r'\s+', ' ', match.strip())
        cleaned_matches.append(cleaned)
    
    return cleaned_matches


def is_likely_header_footer(img: Image.Image, min_size: int = 8000) -> bool:
    """
    Détermine si une image est probablement un en-tête ou pied de page
    basé sur sa taille et ses dimensions.
    """
    try:
        width, height = img.size
        
        # Calculer la taille approximative en bytes (estimation)
        estimated_size = width * height * 3  # RGB approximation
        
        # Critères pour filtrer en-têtes/pieds de page
        if estimated_size < min_size:  # Trop petite
            return True
        if height < 30:  # Trop fine (bandeau)
            return True
        if width < 80:  # Trop étroite
            return True
        if width > height * 15:  # Trop allongée horizontalement (bandeau)
            return True
        if height > width * 5:  # Trop allongée verticalement
            return True
        if width < 150 and height < 150:  # Très petite image (logo, etc.)
            return True
            
        return False
    except Exception:
        return True


def extract_images_from_page(page, page_num: int, pdf_path: Path, reader: PdfReader) -> list[tuple[int, str, Path, str]]:
    """
    Extrait les images d'une page PDF spécifique.
    """
    images = []
    
    try:
        # Extraire le texte de la page
        page_text = page.extract_text() or ""
        
        # Extraire les légendes
        graph_captions = extract_captions(page_text, "Graphique")
        table_captions = extract_captions(page_text, "Tableau")
        all_captions = graph_captions + table_captions
        
        # Obtenir les objets image de la page
        if "/Resources" not in page or "/XObject" not in page["/Resources"]:
            return images
            
        xobjects = page["/Resources"]["/XObject"]
        if not isinstance(xobjects, dict):
            return images
        
        valid_images = []
        
        # Traiter chaque objet image
        for obj_name, obj_ref in xobjects.items():
            try:
                obj = obj_ref.get_object() if isinstance(obj_ref, IndirectObject) else obj_ref
                
                if obj.get("/Subtype") != "/Image":
                    continue
                
                # Extraire les données de l'image
                img_data = obj.get_data()
                if not img_data:
                    continue
                
                # Créer l'image PIL
                img = Image.open(io.BytesIO(img_data))
                
                # Convertir en RGB si nécessaire
                if img.mode != "RGB":
                    img = img.convert("RGB")
                
                # Filtrer les en-têtes/pieds de page
                if not is_likely_header_footer(img):
                    valid_images.append(img)
                
            except Exception as e:
                logger.debug(f"Erreur lors du traitement de l'image {obj_name}: {e}")
                continue
        
        # Associer les images aux légendes et sauvegarder
        for idx, img in enumerate(valid_images):
            caption = all_captions[idx] if idx < len(all_captions) else ""
            title = extract_title_from_caption(caption)
            
            # Déterminer le type (graphique ou tableau)
            if caption.lower().startswith('tableau'):
                type_prefix = "tab"
            else:
                type_prefix = "fig"
            
            # Générer nom de fichier
            if title:
                filename = f"{type_prefix}_{title}_p{page_num}.png"
            else:
                filename = f"{type_prefix}_{pdf_path.stem}_p{page_num}_img{idx + 1}.png"
            
            # Sauvegarder l'image
            out_path = IMAGES_DIR / filename
            out_path.parent.mkdir(parents=True, exist_ok=True)
            
            try:
                img.save(out_path, "PNG", optimize=True)
                images.append((page_num, filename, out_path, caption))
                logger.debug(f"Image sauvegardée: {filename}")
            except Exception as e:
                logger.error(f"Erreur lors de la sauvegarde de {filename}: {e}")
                continue
    
    except Exception as e:
        logger.error(f"Erreur lors du traitement de la page {page_num}: {e}")
    
    return images


def extract_all_images(pdf_path: Path) -> list[tuple[int, str, Path, str]]:
    """
    Extrait toutes les images d'un PDF avec filtrage amélioré.
    """
    all_images = []
    
    try:
        reader = PdfReader(str(pdf_path))
        logger.info(f"Traitement de {len(reader.pages)} pages dans {pdf_path.name}")
        
        for page_num, page in enumerate(reader.pages, start=1):
            page_images = extract_images_from_page(page, page_num, pdf_path, reader)
            all_images.extend(page_images)
            
            if page_images:
                logger.info(f"  Page {page_num}: {len(page_images)} images extraites")
    
    except Exception as e:
        logger.error(f"Erreur lors de l'ouverture du PDF {pdf_path}: {e}")
    
    return all_images


def generate_charts_index(pdf_source: Path) -> None:
    """
    Extrait et indexe tous les graphiques et tableaux comme images.
    Génère un seul CSV: charts_index.csv
    """
    IMAGES_DIR.mkdir(parents=True, exist_ok=True)
    
    # Déterminer les PDFs à traiter
    if pdf_source.is_file():
        pdf_files = [pdf_source]
    else:
        pdf_files = list(pdf_source.rglob("*.pdf"))
    
    logger.info(f"Traitement de {len(pdf_files)} fichiers PDF")
    
    total_images = 0
    successful_pdfs = 0
    
    with INDEX_IMAGES_CSV.open("w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["image_id", "pdf_path", "page", "image_path", "caption", "title", "type"])
        
        for pdf_path in pdf_files:
            print(f"Traitement de {pdf_path.name}...")
            
            try:
                extracted_items = extract_all_images(pdf_path)
                
                for page, filename, path, caption in extracted_items:
                    image_id = filename.rsplit('.', 1)[0]  # Sans extension
                    title = extract_title_from_caption(caption)
                    
                    # Déterminer le type
                    img_type = "tableau" if filename.startswith("tab_") else "graphique"
                    
                    writer.writerow([
                        image_id,
                        str(pdf_path),
                        page,
                        str(path),
                        caption,
                        title,
                        img_type
                    ])
                
                if extracted_items:
                    print(f"  ✓ {len(extracted_items)} éléments extraits")
                    total_images += len(extracted_items)
                    successful_pdfs += 1
                else:
                    print(f"  - Aucune image trouvée")
                
            except Exception as e:
                print(f"  ✗ Erreur lors du traitement: {e}")
                continue
    
    print(f"\n[✔] Traitement terminé:")
    print(f"    - PDFs traités avec succès: {successful_pdfs}/{len(pdf_files)}")
    print(f"    - Total d'images extraites: {total_images}")
    print(f"    - Index généré: {INDEX_IMAGES_CSV}")
    print(f"    - Images sauvegardées dans: {IMAGES_DIR}")


def clean_existing_extractions():
    """Supprime les fichiers et dossiers d'extraction existants."""
    import shutil
    
    if IMAGES_DIR.exists():
        shutil.rmtree(IMAGES_DIR)
        print(f"[✔] Dossier {IMAGES_DIR} supprimé")
    
    if INDEX_IMAGES_CSV.exists():
        INDEX_IMAGES_CSV.unlink()
        print(f"[✔] Fichier {INDEX_IMAGES_CSV} supprimé")


def analyze_pdf_structure(pdf_path: Path) -> dict:
    """
    Analyse la structure d'un PDF pour debug.
    """
    info = {
        "pages": 0,
        "total_images": 0,
        "images_per_page": [],
        "captions_found": []
    }
    
    try:
        reader = PdfReader(str(pdf_path))
        info["pages"] = len(reader.pages)
        
        for page_num, page in enumerate(reader.pages, start=1):
            page_text = page.extract_text() or ""
            
            # Compter les images
            xobjects = page.get("/Resources", {}).get("/XObject", {})
            if isinstance(xobjects, dict):
                page_images = sum(1 for obj_ref in xobjects.values() 
                                if (obj_ref.get_object() if isinstance(obj_ref, IndirectObject) else obj_ref).get("/Subtype") == "/Image")
                info["images_per_page"].append(page_images)
                info["total_images"] += page_images
            
            # Extraire les légendes
            captions = extract_captions(page_text, "Graphique") + extract_captions(page_text, "Tableau")
            if captions:
                info["captions_found"].extend([(page_num, cap) for cap in captions])
    
    except Exception as e:
        info["error"] = str(e)
    
    return info


# Exemple d'utilisation
if __name__ == "__main__":
    # Pour analyser un PDF spécifique
    # pdf_file = Path("./test.pdf")
    # structure = analyze_pdf_structure(pdf_file)
    # print(f"Structure: {structure}")
    
    # Pour traiter tous les PDFs
    pdf_folder = Path("/Users/fatousall/Documents/sun-stats/vos_rapports_rgph")  # Ajustez le chemin
    generate_charts_index(pdf_folder)

INFO:__main__:Traitement de 14 fichiers PDF
INFO:__main__:Traitement de 22 pages dans RAPPORT-PRELIMINAIRE-RGPH-5_2023-.pdf


Traitement de RAPPORT-PRELIMINAIRE-RGPH-5_2023-.pdf...


INFO:__main__:  Page 13: 2 images extraites
INFO:__main__:  Page 16: 2 images extraites
INFO:__main__:  Page 22: 1 images extraites
INFO:__main__:Traitement de 31 pages dans Chapitre 11 - HANDICAP-Rapport-Provisoire-RGPH5_juillet2024.pdf


  ✓ 5 éléments extraits
Traitement de Chapitre 11 - HANDICAP-Rapport-Provisoire-RGPH5_juillet2024.pdf...


INFO:__main__:  Page 1: 1 images extraites
INFO:__main__:  Page 2: 1 images extraites
INFO:__main__:  Page 18: 1 images extraites
INFO:__main__:  Page 19: 1 images extraites
INFO:__main__:  Page 31: 1 images extraites
INFO:__main__:Traitement de 43 pages dans Chapitre 8- HABITAT-Rapport-Provisoire-RGPH5_juillet2024.pdf


...

  ✓ 3 éléments extraits
Traitement de Chapitre 9- MENAGES-Rapport-Provisoire-RGPH5_juillet2024.pdf...


INFO:__main__:  Page 1: 1 images extraites
INFO:__main__:  Page 2: 2 images extraites
INFO:__main__:  Page 3: 1 images extraites
INFO:__main__:  Page 4: 1 images extraites
INFO:__main__:  Page 5: 1 images extraites
INFO:__main__:  Page 6: 1 images extraites
INFO:__main__:  Page 7: 1 images extraites
INFO:__main__:  Page 8: 1 images extraites
INFO:__main__:  Page 9: 1 images extraites
INFO:__main__:  Page 10: 1 images extraites
INFO:__main__:  Page 11: 1 images extraites
INFO:__main__:  Page 12: 1 images extraites
INFO:__main__:  Page 13: 1 images extraites
INFO:__main__:  Page 14: 1 images extraites
INFO:__main__:  Page 15: 1 images extraites
INFO:__main__:  Page 16: 1 images extraites
INFO:__main__:  Page 17: 1 images extraites
INFO:__main__:  Page 18: 1 images extraites
INFO:__main__:  Page 19: 1 images extraites
INFO:__main__:  Page 20: 1 images extraites
INFO:__main__:  Page 21: 1 images extraites
INFO:__main__:  Page 22: 1 images extraites
INFO:__main__:  Page 23: 1 images extrait

  ✓ 33 éléments extraits
Traitement de Chapitre 4 - FECONDITE-NATALITE-Rapport-Provisoire-RGPH5_juillet2024_0.pdf...


INFO:__main__:  Page 1: 1 images extraites
INFO:__main__:  Page 2: 2 images extraites
INFO:__main__:  Page 3: 1 images extraites
INFO:__main__:  Page 4: 1 images extraites
INFO:__main__:  Page 5: 1 images extraites
INFO:__main__:  Page 6: 1 images extraites
INFO:__main__:  Page 7: 1 images extraites
INFO:__main__:  Page 8: 1 images extraites
INFO:__main__:  Page 9: 1 images extraites
INFO:__main__:  Page 10: 1 images extraites
INFO:__main__:  Page 11: 1 images extraites
INFO:__main__:  Page 12: 1 images extraites
INFO:__main__:  Page 13: 1 images extraites
INFO:__main__:  Page 14: 1 images extraites
INFO:__main__:  Page 15: 1 images extraites
INFO:__main__:  Page 16: 1 images extraites
INFO:__main__:  Page 17: 2 images extraites
INFO:__main__:  Page 18: 1 images extraites
INFO:__main__:  Page 19: 2 images extraites
INFO:__main__:  Page 20: 1 images extraites
INFO:__main__:  Page 21: 2 images extraites
INFO:__main__:  Page 22: 1 images extraites
INFO:__main__:  Page 23: 2 images extrait

  ✓ 63 éléments extraits

[✔] Traitement terminé:
    - PDFs traités avec succès: 14/14
    - Total d'images extraites: 468
    - Index généré: charts_index.csv
    - Images sauvegardées dans: images


In [4]:
# Exemple d'utilisation
if __name__ == "__main__":
    # Pour déboguer une page spécifique qui pose problème
    # pdf_file = Path("./votre_pdf_avec_petit_tableau.pdf")
    # debug_info = debug_page_images(pdf_file, 1)  # Page 1
    # print(f"Images trouvées: {debug_info['processing_summary']['total_images']}")
    # print(f"Images conservées: {debug_info['processing_summary']['kept_images']}")
    # for img in debug_info['images_found']:
    #     print(f"  - {img['object_name']}: {img['dimensions']} - {img['content_type']} - Gardée: {img['would_be_kept']}")
    
    # Pour analyser les légendes dans un PDF spécifique et voir le filtrage
    # pdf_file = Path("./RAPPORT-PRELIMINAIRE-RGPH-5_2023-.pdf")
    # analysis = analyze_captions_in_pdf(pdf_file)
    # print(f"Légendes valides trouvées: {len(analysis['valid_captions'])}")
    # print(f"Légendes filtrées (titre PDF): {len(analysis['pdf_title_filtered'])}")
    # print(f"Légendes invalides: {len(analysis['invalid_captions'])}")
    
    # Pour traiter tous les PDFs (mode titre uniquement + exclusion titre PDF + petits tableaux)
    pdf_folder = Path("/Users/fatousall/Documents/sun-stats/vos_rapports_rgph")
    generate_charts_index(pdf_folder)# shared/utils.py
"""
Utilitaires partagés pour l'extraction d'images (graphiques) et de tables depuis les PDF
et création des index CSV correspondants avec légendes.
Version améliorée - extraction uniquement des éléments avec titre.
"""
import csv
import io
import re
from pathlib import Path
import logging
from typing import List, Tuple, Optional, Dict

from PyPDF2 import PdfReader
from PyPDF2.generic import IndirectObject
from PIL import Image

# Dossiers et fichiers fixes à la racine
IMAGES_DIR = Path("./images")
INDEX_IMAGES_CSV = Path("./charts_index.csv")

# Configuration du logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


def extract_title_from_caption(caption: str) -> str:
    """
    Extrait le titre principal d'une légende en supprimant le préfixe et les caractères spéciaux.
    Ex: "Graphique 2 : Rapport de masculinité de la population" -> "Rapport_de_masculinite_de_la_population"
    """
    if not caption:
        return ""
    
    # Supprimer le préfixe (Graphique X:, Tableau Y:, etc.)
    title = re.sub(r'^(Graphique|Tableau)\s+[\w\-]+\s*:\s*', '', caption, flags=re.IGNORECASE)
    
    # Nettoyer et formater pour nom de fichier
    title = re.sub(r'[^\w\s\-]', '', title)  # Garder lettres, chiffres, espaces, tirets
    title = re.sub(r'\s+', '_', title.strip())  # Remplacer espaces par underscores
    title = title.lower()  # Minuscules pour cohérence
    
    return title[:50]  # Limiter la longueur


def extract_captions_advanced(text: str) -> Dict[str, List[str]]:
    """
    Extrait les légendes de graphiques et tableaux avec patterns améliorés.
    Retourne un dictionnaire avec les types et leurs légendes respectives.
    """
    if not text:
        return {"graphiques": [], "tableaux": []}
    
    # Patterns plus robustes pour différents formats
    patterns = {
        "graphiques": [
            r"^Graphique\s+[\d\w\-\.]+\s*:\s*.+?(?=\n\n|\n[A-Z]|\nGraphique|\nTableau|\Z)",
            r"^Figure\s+[\d\w\-\.]+\s*[:\-]\s*.+?(?=\n\n|\n[A-Z]|\nFigure|\nTableau|\Z)",
            r"^Fig\.\s*[\d\w\-\.]+\s*[:\-]\s*.+?(?=\n\n|\n[A-Z]|\nFig\.|\nTableau|\Z)"
        ],
        "tableaux": [
            r"^Tableau\s+[\d\w\-\.]+\s*:\s*.+?(?=\n\n|\n[A-Z]|\nTableau|\nGraphique|\Z)",
            r"^Table\s+[\d\w\-\.]+\s*[:\-]\s*.+?(?=\n\n|\n[A-Z]|\nTable|\nGraphique|\Z)"
        ]
    }
    
    results = {"graphiques": [], "tableaux": []}
    
    for category, pattern_list in patterns.items():
        for pattern in pattern_list:
            matches = re.findall(pattern, text, re.IGNORECASE | re.MULTILINE | re.DOTALL)
            
            # Nettoyer les matches
            for match in matches:
                cleaned = re.sub(r'\s+', ' ', match.strip())
                if cleaned not in results[category]:  # Éviter les doublons
                    results[category].append(cleaned)
    
    return results


def has_valid_title(caption: str, pdf_path: Path = None) -> bool:
    """
    Vérifie si une légende contient un titre valide (non vide après nettoyage).
    Exclut les titres qui ressemblent au nom du fichier PDF.
    """
    if not caption:
        return False
    
    # Extraire et nettoyer le titre
    title = extract_title_from_caption(caption)
    
    # Critères de validation de base
    if not title:
        return False
    if len(title) < 3:  # Titre trop court
        return False
    if title.lower() in ['sans_titre', 'untitled', 'no_title', 'titre', 'title']:
        return False
    
    # Vérifier que le titre n'est pas similaire au nom du PDF
    if pdf_path:
        pdf_name_parts = extract_pdf_name_parts(pdf_path)
        if is_title_similar_to_pdf_name(title, pdf_name_parts):
            return False
    
    return True


def extract_pdf_name_parts(pdf_path: Path) -> List[str]:
    """
    Extrait les parties significatives du nom du fichier PDF pour comparaison.
    """
    if not pdf_path:
        return []
    
    # Obtenir le nom sans extension
    pdf_name = pdf_path.stem.lower()
    
    # Nettoyer et découper en parties
    # Supprimer les caractères spéciaux et garder les mots
    cleaned_name = re.sub(r'[^\w\s\-]', ' ', pdf_name)
    
    # Découper en mots significatifs (> 2 caractères)
    parts = [part.strip() for part in re.split(r'[\s\-_]+', cleaned_name) 
             if len(part.strip()) > 2]
    
    return parts


def is_title_similar_to_pdf_name(title: str, pdf_name_parts: List[str]) -> bool:
    """
    Vérifie si un titre ressemble trop au nom du fichier PDF.
    """
    if not title or not pdf_name_parts:
        return False
    
    title_lower = title.lower()
    title_words = set(re.split(r'[\s\-_]+', title_lower))
    
    # Supprimer les mots très courts ou communs
    common_words = {'de', 'du', 'des', 'le', 'la', 'les', 'et', 'ou', 'pour', 'par', 'sur', 'avec', 'dans', 'sans', 'selon', 'entre', 'vers', 'chez', 'sous', 'depuis', 'pendant', 'avant', 'après', 'chapitre', 'rapport', 'provisoire', 'preliminaire', 'rgph', 'juillet', '2024', '2023'}
    title_words = {word for word in title_words if len(word) > 2 and word not in common_words}
    
    pdf_words = {word.lower() for word in pdf_name_parts if len(word) > 2 and word.lower() not in common_words}
    
    if not title_words or not pdf_words:
        return False
    
    # Calculer le pourcentage de chevauchement
    overlap = title_words.intersection(pdf_words)
    overlap_ratio = len(overlap) / min(len(title_words), len(pdf_words))
    
    # Si plus de 60% des mots se chevauchent, c'est probablement le titre du PDF
    return overlap_ratio > 0.6


def categorize_caption(caption: str, pdf_path: Path = None) -> Optional[str]:
    """
    Détermine la catégorie d'une légende (graphique ou tableau).
    Retourne None si la légende n'est pas valide.
    """
    if not has_valid_title(caption, pdf_path):
        return None
    
    caption_lower = caption.lower()
    
    if any(word in caption_lower for word in ['graphique', 'figure', 'fig.']):
        return 'graphique'
    elif any(word in caption_lower for word in ['tableau', 'table']):
        return 'tableau'
    
    return None


def detect_table_structure(img: Image.Image) -> Dict:
    """
    Détecte si une image contient une structure de tableau
    en analysant les lignes et les patterns géométriques.
    """
    try:
        import numpy as np
        from PIL import ImageFilter
        
        # Convertir en niveaux de gris
        gray = img.convert('L')
        
        # Appliquer un filtre pour détecter les lignes
        edges = gray.filter(ImageFilter.FIND_EDGES)
        
        # Convertir en array numpy
        img_array = np.array(gray)
        edges_array = np.array(edges)
        
        width, height = img.size
        
        # Analyser les lignes horizontales (typiques des tableaux)
        horizontal_lines = 0
        for y in range(0, height, max(1, height // 20)):  # Échantillonner 20 lignes
            if y < height:
                row = img_array[y, :]
                # Détecter les variations (lignes de séparation)
                row_diff = np.diff(row.astype(int))
                if np.std(row_diff) > 10:  # Variation significative
                    horizontal_lines += 1
        
        # Analyser les lignes verticales
        vertical_lines = 0
        for x in range(0, width, max(1, width // 20)):  # Échantillonner 20 colonnes
            if x < width:
                col = img_array[:, x]
                col_diff = np.diff(col.astype(int))
                if np.std(col_diff) > 10:
                    vertical_lines += 1
        
        # Calculer la densité de contour (tableaux ont plus de contours)
        edge_density = np.sum(edges_array > 50) / (width * height)
        
        # Analyser les zones uniformes vs structurées
        # Diviser l'image en grille 4x4 et analyser la variance de chaque zone
        zone_variances = []
        grid_size = 4
        zone_width = width // grid_size
        zone_height = height // grid_size
        
        for i in range(grid_size):
            for j in range(grid_size):
                x1, y1 = i * zone_width, j * zone_height
                x2, y2 = min((i + 1) * zone_width, width), min((j + 1) * zone_height, height)
                
                if x2 > x1 and y2 > y1:
                    zone = img_array[y1:y2, x1:x2]
                    if zone.size > 0:
                        zone_variances.append(np.var(zone))
        
        avg_zone_variance = np.mean(zone_variances) if zone_variances else 0
        
        structure_analysis = {
            "horizontal_line_density": horizontal_lines / max(1, height // 20),
            "vertical_line_density": vertical_lines / max(1, width // 20),
            "edge_density": edge_density,
            "avg_zone_variance": avg_zone_variance,
            "has_table_structure": False,
            "structure_score": 0
        }
        
        # Score de structure de tableau
        structure_score = 0
        
        # Les tableaux ont des lignes horizontales régulières
        if structure_analysis["horizontal_line_density"] > 0.3:
            structure_score += 2
        
        # Les tableaux ont aussi des lignes verticales
        if structure_analysis["vertical_line_density"] > 0.2:
            structure_score += 2
        
        # Density de contours modérée (ni trop lisse ni trop chaotique)
        if 0.05 < edge_density < 0.3:
            structure_score += 1
        
        # Variance modérée et régulière entre zones
        if 100 < avg_zone_variance < 2000:
            structure_score += 1
        
        structure_analysis["structure_score"] = structure_score
        structure_analysis["has_table_structure"] = structure_score >= 3
        
        return structure_analysis
        
    except Exception as e:
        return {
            "error": str(e),
            "has_table_structure": False,
            "structure_score": 0
        }


def detect_header_banner(img: Image.Image) -> bool:
    """
    Détecte spécifiquement les en-têtes et bannières
    comme celui de votre exemple (bleu avec texte centré).
    """
    try:
        width, height = img.size
        
        # Critères géométriques pour bannières
        aspect_ratio = width / height if height > 0 else 0
        
        # Les bannières sont généralement très larges et plates
        is_banner_shape = (
            aspect_ratio > 8 or  # Très allongée horizontalement
            (width > 500 and height < 100) or  # Large et plate
            (height < 60 and width > 300)  # Plate et assez large
        )
        
        if not is_banner_shape:
            return False
        
        # Analyser la distribution des couleurs
        img_rgb = img.convert('RGB')
        
        # Compter les couleurs dominantes
        colors = img_rgb.getcolors(maxcolors=256*256*256)
        if not colors:
            return False
        
        # Trier par fréquence
        colors.sort(key=lambda x: x[0], reverse=True)
        
        # Les bannières ont souvent une couleur dominante (fond coloré)
        total_pixels = width * height
        dominant_color_ratio = colors[0][0] / total_pixels if colors else 0
        
        # Si une couleur représente plus de 30% de l'image, c'est probablement une bannière
        has_dominant_background = dominant_color_ratio > 0.3
        
        # Analyser la position du texte (bannières = texte centré)
        gray = img.convert('L')
        import numpy as np
        img_array = np.array(gray)
        
        # Détecter les zones de texte (pixels sombres)
        text_pixels = img_array < 150  # Seuil pour texte sombre
        
        if np.any(text_pixels):
            # Trouver le centre de masse du texte
            text_coords = np.where(text_pixels)
            if len(text_coords[0]) > 0:
                text_center_y = np.mean(text_coords[0]) / height
                text_center_x = np.mean(text_coords[1]) / width
                
                # Le texte est-il centré verticalement et horizontalement ?
                is_centered = (0.3 < text_center_y < 0.7) and (0.2 < text_center_x < 0.8)
            else:
                is_centered = False
        else:
            is_centered = False
        
        # Une bannière combine: forme allongée + fond coloré dominant + texte centré
        is_banner = is_banner_shape and has_dominant_background and is_centered
        
        return is_banner
        
    except Exception:
        return False


def is_meaningful_image(img: Image.Image, min_complexity: float = 0.08) -> bool:
    """
    Vérifie si une image semble contenir du contenu significatif.
    Version assouplie pour capturer les petits tableaux.
    """
    try:
        width, height = img.size
        
        # Filtres de base sur les dimensions (assouplis)
        if width < 120 or height < 80:  # Réduit pour petits tableaux
            return False
        
        # Convertir en niveaux de gris pour analyser la complexité
        gray_img = img.convert('L')
        
        # Calculer l'écart-type des pixels (mesure de variabilité)
        import statistics
        pixels = list(gray_img.getdata())
        
        if len(pixels) == 0:
            return False
        
        # Si tous les pixels sont presque identiques, ce n'est pas intéressant
        unique_values = len(set(pixels))
        if unique_values < 12:  # Réduit le seuil
            return False
        
        # Calculer la variabilité
        try:
            std_dev = statistics.stdev(pixels)
            mean_val = statistics.mean(pixels)
            
            # Coefficient de variation (assoupli)
            cv = std_dev / mean_val if mean_val > 0 else 0
            
            # Critères plus souples pour capturer les petits tableaux
            if cv < min_complexity:
                return False
            
            # Vérifier la distribution des valeurs de gris (assouplie)
            pixel_range = max(pixels) - min(pixels)
            if pixel_range < 25:  # Contraste plus faible accepté
                return False
            
            return True
            
        except:
            return True  # En cas d'erreur, on garde l'image par défaut
            
    except Exception:
        return True


def analyze_image_content(img: Image.Image) -> Dict:
    """
    Analyse le contenu d'une image pour déterminer si c'est un tableau/graphique valide.
    Version avec détection de structure de tableau.
    """
    try:
        width, height = img.size
        
        # Analyser la distribution des couleurs
        gray_img = img.convert('L')
        pixels = list(gray_img.getdata())
        
        import statistics
        
        analysis = {
            "width": width,
            "height": height,
            "area": width * height,
            "aspect_ratio": width / height if height > 0 else 0,
            "pixel_count": len(pixels),
            "unique_values": len(set(pixels)),
            "mean_brightness": statistics.mean(pixels) if pixels else 0,
            "std_dev": 0,
            "contrast_range": 0,
            "is_likely_content": False,
            "content_type": "unknown"
        }
        
        if pixels:
            try:
                analysis["std_dev"] = statistics.stdev(pixels)
                analysis["contrast_range"] = max(pixels) - min(pixels)
            except:
                pass
        
        # Détecter la structure de tableau
        table_structure = detect_table_structure(img)
        analysis.update(table_structure)
        
        # Détecter les bannières/en-têtes
        is_banner = detect_header_banner(img)
        analysis["is_banner"] = is_banner
        
        # Critères de base (obligatoires)
        base_criteria = [
            width > 120,
            height > 80,
            analysis["unique_values"] > 15,
            analysis["contrast_range"] > 30,
            1 < analysis["aspect_ratio"] < 12,
        ]
        
        # Critères pour tableaux (priorité haute si structure détectée)
        table_criteria = [
            analysis.get("has_table_structure", False),  # Structure de tableau détectée
            analysis.get("structure_score", 0) >= 3,     # Score de structure élevé
            analysis["area"] > 15000,                     # Taille raisonnable
            2 < analysis["aspect_ratio"] < 8,            # Proportion tableau
            analysis["std_dev"] > 15,                     # Variabilité
        ]
        
        # Critères pour graphiques/contenus larges
        graphic_criteria = [
            analysis["area"] > 25000,
            analysis["std_dev"] > 20,
            analysis["unique_values"] > 25,
            analysis["contrast_range"] > 50,
            analysis["aspect_ratio"] < 6,
        ]
        
        # Logique de décision
        has_base_quality = sum(base_criteria) >= 4
        is_table = sum(table_criteria) >= 3
        is_graphic = sum(graphic_criteria) >= 3
        
        if is_banner:
            analysis["is_likely_content"] = False
            analysis["content_type"] = "banner/header"
        elif has_base_quality and is_table:
            analysis["is_likely_content"] = True
            analysis["content_type"] = "table"
        elif has_base_quality and is_graphic:
            analysis["is_likely_content"] = True
            analysis["content_type"] = "graphic"
        elif has_base_quality and analysis["area"] > 12000:  # Petits contenus valides
            analysis["is_likely_content"] = True
            analysis["content_type"] = "small_content"
        else:
            analysis["is_likely_content"] = False
            analysis["content_type"] = "filtered"
        
        return analysis
        
    except Exception as e:
        return {"error": str(e), "is_likely_content": False, "content_type": "error"}


def extract_images_from_page(page, page_num: int, pdf_path: Path, reader: PdfReader) -> List[Tuple[int, str, Path, str, str]]:
    """
    Extrait les images d'une page PDF spécifique, uniquement celles avec des titres valides.
    Version améliorée avec analyse de contenu pour éviter les pieds de page.
    """
    valid_images = []
    
    try:
        # Extraire le texte de la page
        page_text = page.extract_text() or ""
        
        # Extraire les légendes avec la nouvelle méthode
        captions_data = extract_captions_advanced(page_text)
        all_captions = captions_data["graphiques"] + captions_data["tableaux"]
        
        # Filtrer les légendes pour ne garder que celles avec des titres valides
        valid_captions = []
        for caption in all_captions:
            category = categorize_caption(caption, pdf_path)
            if category:
                valid_captions.append((caption, category))
        
        # Si aucune légende valide trouvée, on arrête
        if not valid_captions:
            logger.debug(f"Page {page_num}: Aucune légende valide trouvée")
            return valid_images
        
        # Obtenir les objets image de la page
        if "/Resources" not in page or "/XObject" not in page["/Resources"]:
            return valid_images
            
        xobjects = page["/Resources"]["/XObject"]
        if not isinstance(xobjects, dict):
            return valid_images
        
        extracted_images = []
        
        # Traiter chaque objet image avec analyse détaillée
        for obj_name, obj_ref in xobjects.items():
            try:
                obj = obj_ref.get_object() if isinstance(obj_ref, IndirectObject) else obj_ref
                
                if obj.get("/Subtype") != "/Image":
                    continue
                
                # Extraire les données de l'image
                img_data = obj.get_data()
                if not img_data:
                    continue
                
                # Créer l'image PIL
                img = Image.open(io.BytesIO(img_data))
                
                # Convertir en RGB si nécessaire
                if img.mode != "RGB":
                    img = img.convert("RGB")
                
                # Analyser le contenu de l'image avec détection de structure
                content_analysis = analyze_image_content(img)
                
                # Filtres intelligents : privilégier les tableaux détectés
                is_header_footer = content_analysis.get("is_banner", False)
                has_table_structure = content_analysis.get("has_table_structure", False)
                
                # Critères d'acceptation plus intelligents
                should_keep = (
                    not is_header_footer and 
                    is_meaningful_image(img) and
                    (content_analysis["is_likely_content"] or has_table_structure)
                )
                
                if should_keep:
                    extracted_images.append({
                        "image": img,
                        "analysis": content_analysis,
                        "object_name": obj_name
                    })
                    logger.debug(f"Image conservée: {obj_name} - Type: {content_analysis['content_type']} - Structure tableau: {has_table_structure} - Taille: {content_analysis['area']}")
                else:
                    logger.debug(f"Image filtrée: {obj_name} - Type: {content_analysis.get('content_type', 'unknown')} - Bannière: {is_header_footer} - Taille: {content_analysis.get('area', 0)}")
                
            except Exception as e:
                logger.debug(f"Erreur lors du traitement de l'image {obj_name}: {e}")
                continue
        
        # Trier les images : priorité aux tableaux détectés, puis par taille
        extracted_images.sort(key=lambda x: (
            -int(x["analysis"].get("has_table_structure", False)),  # Tableaux en premier (True = 1, False = 0, donc -1 vs 0)
            -x["analysis"]["area"]  # Puis par taille décroissante
        ))
        
        # Associer les meilleures images aux légendes valides
        max_items = min(len(extracted_images), len(valid_captions))
        
        for idx in range(max_items):
            img_data = extracted_images[idx]
            img = img_data["image"]
            caption, category = valid_captions[idx]
            
            title = extract_title_from_caption(caption)
            
            # Déterminer le préfixe selon la catégorie
            type_prefix = "tab" if category == "tableau" else "fig"
            
            # Générer nom de fichier avec le titre valide + nom du PDF
            pdf_name_clean = re.sub(r'[^\w\-]', '_', pdf_path.stem)[:30]  # Nettoyer et limiter
            filename = f"{type_prefix}_{title}_{pdf_name_clean}_p{page_num}.png"
            
            # Sauvegarder l'image
            out_path = IMAGES_DIR / filename
            out_path.parent.mkdir(parents=True, exist_ok=True)
            
            try:
                img.save(out_path, "PNG", optimize=True)
                valid_images.append((page_num, filename, out_path, caption, category))
                structure_info = f"(structure: {img_data['analysis'].get('structure_score', 0)}/6)" if img_data['analysis'].get('has_table_structure') else ""
                logger.info(f"Image avec titre sauvegardée: {filename} (type: {img_data['analysis']['content_type']}, taille: {img_data['analysis']['area']}, PDF: {pdf_path.name}) {structure_info}")
            except Exception as e:
                logger.error(f"Erreur lors de la sauvegarde de {filename}: {e}")
                continue
    
    except Exception as e:
        logger.error(f"Erreur lors du traitement de la page {page_num}: {e}")
    
    return valid_images


def extract_all_images(pdf_path: Path) -> List[Tuple[int, str, Path, str, str]]:
    """
    Extrait toutes les images d'un PDF avec filtrage pour ne garder que celles avec des titres.
    """
    all_images = []
    
    try:
        reader = PdfReader(str(pdf_path))
        logger.info(f"Traitement de {len(reader.pages)} pages dans {pdf_path.name}")
        
        for page_num, page in enumerate(reader.pages, start=1):
            page_images = extract_images_from_page(page, page_num, pdf_path, reader)
            all_images.extend(page_images)
            
            if page_images:
                logger.info(f"  Page {page_num}: {len(page_images)} images avec titre extraites")
            else:
                logger.debug(f"  Page {page_num}: Aucune image avec titre valide")
    
    except Exception as e:
        logger.error(f"Erreur lors de l'ouverture du PDF {pdf_path}: {e}")
    
    return all_images


def generate_charts_index(pdf_source: Path) -> None:
    """
    Extrait et indexe uniquement les graphiques et tableaux avec des titres valides.
    """
    IMAGES_DIR.mkdir(parents=True, exist_ok=True)
    
    # Déterminer les PDFs à traiter
    if pdf_source.is_file():
        pdf_files = [pdf_source]
    else:
        pdf_files = list(pdf_source.rglob("*.pdf"))
    
    logger.info(f"Traitement de {len(pdf_files)} fichiers PDF")
    
    total_images = 0
    successful_pdfs = 0
    skipped_items = 0
    
    with INDEX_IMAGES_CSV.open("w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["image_id", "pdf_path", "pdf_name", "page", "image_path", "caption", "title", "type"])
        
        for pdf_path in pdf_files:
            print(f"Traitement de {pdf_path.name}...")
            
            try:
                extracted_items = extract_all_images(pdf_path)
                
                for page, filename, path, caption, category in extracted_items:
                    image_id = filename.rsplit('.', 1)[0]  # Sans extension
                    title = extract_title_from_caption(caption)
                    
                    writer.writerow([
                        image_id,
                        str(pdf_path),
                        pdf_path.name,  # Nouveau: nom du fichier PDF
                        page,
                        str(path),
                        caption,
                        title,
                        category
                    ])
                
                if extracted_items:
                    print(f"  ✓ {len(extracted_items)} éléments avec titre extraits")
                    total_images += len(extracted_items)
                    successful_pdfs += 1
                else:
                    print(f"  - Aucune image avec titre valide trouvée")
                
            except Exception as e:
                print(f"  ✗ Erreur lors du traitement: {e}")
                continue
    
    print(f"\n[✔] Traitement terminé (mode titre uniquement - exclusion titres PDF):")
    print(f"    - PDFs traités avec succès: {successful_pdfs}/{len(pdf_files)}")
    print(f"    - Total d'images avec titre unique extraites: {total_images}")
    print(f"    - Index généré: {INDEX_IMAGES_CSV}")
    print(f"    - Images sauvegardées dans: {IMAGES_DIR}")
    print(f"    - Note: Les titres similaires aux noms de PDF sont exclus")


def analyze_captions_in_pdf(pdf_path: Path) -> Dict:
    """
    Analyse les légendes dans un PDF pour débugger et comprendre la structure.
    """
    analysis = {
        "total_pages": 0,
        "pages_with_captions": 0,
        "valid_captions": [],
        "invalid_captions": [],
        "pdf_title_filtered": [],  # Nouvelles entrées filtrées par similitude avec titre PDF
        "caption_patterns": {"graphiques": 0, "tableaux": 0}
    }
    
    try:
        reader = PdfReader(str(pdf_path))
        analysis["total_pages"] = len(reader.pages)
        
        # Extraire les parties du nom du PDF pour comparaison
        pdf_name_parts = extract_pdf_name_parts(pdf_path)
        
        for page_num, page in enumerate(reader.pages, start=1):
            page_text = page.extract_text() or ""
            
            captions_data = extract_captions_advanced(page_text)
            all_captions = captions_data["graphiques"] + captions_data["tableaux"]
            
            if all_captions:
                analysis["pages_with_captions"] += 1
                
                for caption in all_captions:
                    title = extract_title_from_caption(caption)
                    
                    # Vérifier si filtré par similitude avec le titre PDF
                    if title and pdf_name_parts and is_title_similar_to_pdf_name(title, pdf_name_parts):
                        analysis["pdf_title_filtered"].append({
                            "page": page_num,
                            "caption": caption,
                            "title": title,
                            "reason": "Similaire au titre du PDF"
                        })
                        continue
                    
                    category = categorize_caption(caption, pdf_path)
                    if category:
                        analysis["valid_captions"].append({
                            "page": page_num,
                            "caption": caption,
                            "category": category,
                            "title": title
                        })
                        analysis["caption_patterns"][f"{category}s"] += 1
                    else:
                        analysis["invalid_captions"].append({
                            "page": page_num,
                            "caption": caption,
                            "title": title,
                            "reason": "Pas de titre valide ou autre critère"
                        })
    
    except Exception as e:
        analysis["error"] = str(e)
    
    return analysis


def clean_existing_extractions():
    """Supprime les fichiers et dossiers d'extraction existants."""
    import shutil
    
    if IMAGES_DIR.exists():
        shutil.rmtree(IMAGES_DIR)
        print(f"[✔] Dossier {IMAGES_DIR} supprimé")
    
    if INDEX_IMAGES_CSV.exists():
        INDEX_IMAGES_CSV.unlink()
        print(f"[✔] Fichier {INDEX_IMAGES_CSV} supprimé")


# Exemple d'utilisation
if __name__ == "__main__":
    # Pour analyser les légendes dans un PDF spécifique
    # pdf_file = Path("./test.pdf")
    # analysis = analyze_captions_in_pdf(pdf_file)
    # print(f"Analyse des légendes: {analysis}")
    
    # Pour traiter tous les PDFs (mode titre uniquement)
    pdf_folder = Path("/Users/fatousall/Documents/sun-stats/vos_rapports_rgph")
    generate_charts_index(pdf_folder)

/Users/fatousall/anaconda3/lib/python3.11/site-packages/camelot/parsers/base.py:238: UserWarning: No tables found in table area (0, 0, 595.276, 841.89)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
/Users/fatousall/anaconda3/lib/python3.11/site-packages/camelot/parsers/base.py:238: UserWarning: No tables found in table area (46.6929, 565.3519000000001, 541.0133, 786.88746)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
/Users/fatousall/anaconda3/lib/python3.11/site-packages/camelot/parsers/base.py:238: UserWarning: No tables found in table area (46.6929, 535.6221, 541.0252999999998, 742.9593380952381)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
/Users/fatousall/anaconda3/lib/python3.11/site-packages/camelot/parsers/base.py:238: UserWarning: No tables found in table area (58.031499999999994, 246.04399999999998, 552.3699, 442.0376263736264)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, u

[✔] Images indexées -> charts_index.csv
[✔] Tables indexées -> tables_index.csv
Traitement de RAPPORT-PRELIMINAIRE-RGPH-5_2023-.pdf...


INFO:__main__:Image avec titre sauvegardée: fig_pyramide_des_âges_de_la_population_du_sénégal_en_2_RAPPORT-PRELIMINAIRE-RGPH-5_20_p16.png (type: table, taille: 611072, PDF: RAPPORT-PRELIMINAIRE-RGPH-5_2023-.pdf) (structure: 3/6)
INFO:__main__:Image avec titre sauvegardée: fig_rapport_de_masculinité_de_la_population_du_sénégal_RAPPORT-PRELIMINAIRE-RGPH-5_20_p16.png (type: table, taille: 313348, PDF: RAPPORT-PRELIMINAIRE-RGPH-5_2023-.pdf) (structure: 4/6)
INFO:__main__:  Page 16: 2 images avec titre extraites
INFO:__main__:Traitement de 31 pages dans Chapitre 11 - HANDICAP-Rapport-Provisoire-RGPH5_juillet2024.pdf


  ✓ 2 éléments avec titre extraits
Traitement de Chapitre 11 - HANDICAP-Rapport-Provisoire-RGPH5_juillet2024.pdf...


INFO:__main__:Image avec titre sauvegardée: fig_prévalence_du_handicap_par_milieu_de_résidence_sel_Chapitre_11_-_HANDICAP-Rapport_p18.png (type: table, taille: 289224, PDF: Chapitre 11 - HANDICAP-Rapport-Provisoire-RGPH5_juillet2024.pdf) (structure: 5/6)
INFO:__main__:  Page 18: 1 images avec titre extraites
INFO:__main__:Image avec titre sauvegardée: fig_prévalence_du_handicap_selon_la_région_Chapitre_11_-_HANDICAP-Rapport_p19.png (type: table, taille: 569088, PDF: Chapitre 11 - HANDICAP-Rapport-Provisoire-RGPH5_juillet2024.pdf) (structure: 6/6)
INFO:__main__:  Page 19: 1 images avec titre extraites
INFO:__main__:Traitement de 43 pages dans Chapitre 8- HABITAT-Rapport-Provisoire-RGPH5_juillet2024.pdf


...

  - Aucune image avec titre valide trouvée
Traitement de Chapitre 9- MENAGES-Rapport-Provisoire-RGPH5_juillet2024.pdf...


INFO:__main__:Traitement de 50 pages dans Chapitre 4 - FECONDITE-NATALITE-Rapport-Provisoire-RGPH5_juillet2024_0.pdf


  - Aucune image avec titre valide trouvée
Traitement de Chapitre 4 - FECONDITE-NATALITE-Rapport-Provisoire-RGPH5_juillet2024_0.pdf...


INFO:__main__:Image avec titre sauvegardée: tab_différence_entre_les_naissances_des_12_derniers_mo_Chapitre_4_-_FECONDITE-NATALIT_p17.png (type: table, taille: 346815, PDF: Chapitre 4 - FECONDITE-NATALITE-Rapport-Provisoire-RGPH5_juillet2024_0.pdf) (structure: 6/6)
INFO:__main__:  Page 17: 1 images avec titre extraites
INFO:__main__:Image avec titre sauvegardée: fig_comparaison_des_taux_de_fécondité_déclarés_et_ajus_Chapitre_4_-_FECONDITE-NATALIT_p21.png (type: table, taille: 277398, PDF: Chapitre 4 - FECONDITE-NATALITE-Rapport-Provisoire-RGPH5_juillet2024_0.pdf) (structure: 6/6)
INFO:__main__:  Page 21: 1 images avec titre extraites
INFO:__main__:Image avec titre sauvegardée: fig_taux_de_fécondité_par_âge_selon_le_milieu_de_résid_Chapitre_4_-_FECONDITE-NATALIT_p23.png (type: table, taille: 354178, PDF: Chapitre 4 - FECONDITE-NATALITE-Rapport-Provisoire-RGPH5_juillet2024_0.pdf) (structure: 6/6)
INFO:__main__:  Page 23: 1 images avec titre extraites
INFO:__main__:Image avec titre sauveg

  ✓ 9 éléments avec titre extraits

[✔] Traitement terminé (mode titre uniquement - exclusion titres PDF):
    - PDFs traités avec succès: 6/14
    - Total d'images avec titre unique extraites: 20
    - Index généré: charts_index.csv
    - Images sauvegardées dans: images
    - Note: Les titres similaires aux noms de PDF sont exclus
